# Capstone Project Week 3.3

Importing the modules needed for this project

In [1]:
# Importing and installing the neccesary modules
!conda install -c conda-forge folium=0.5.0 --yes
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
pd.options.mode.chained_assignment = None

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


Different actions to obtain the dataframe required for the exercise. Using Beautiful Soup 4 as the flavor of the read html statement

In [2]:
text=pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',flavor='bs4') # Extracting the data with Beautiful Soup using read_html
df=text[0] # Extracting the first table of the file
df.columns=df.iloc[0] # Reseting the columns' name so the first row becomes the names
df.drop(0,axis=0,inplace=True) # Removing the first row so we don't have duplicates
df.rename(columns={'Postcode':'PostalCode'},inplace=True) # Changing the name of Postcode to PostalCode
df=df[df.Borough != 'Not assigned'] # Dropping the rows whose Borough is not defined
df.reset_index(drop=True,inplace=True) # Reseting the index
for i in df.index:     # Loop to assign the value of the neighbourhood if not assigned to the same Borough.
    if df.Neighbourhood[i] == 'Not assigned':
        df.Neighbourhood[i] = df.Borough[i]
df=df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(lambda Neighbourhood: ', '.join(Neighbourhood)).to_frame() # Grouping the results by Borough and joining the results 
df.reset_index(inplace=True) # Reseting the index again

Extracting the csv files due to several issues with the Geocoder API:

In [3]:
latlng=pd.read_csv('https://cocl.us/Geospatial_data') # Downloading the data from a csv file

Considering that both files are not sorted we are going to create a loop to compare both and join the exact ones

In [4]:
df['Latitude']=0.0      # To define the new columns. 0.0 to determine that we are using floats.
df['Longitude']=0.0     # To define the new columns. 0.0 to determine that we are using floats.
# Comparing both dataframes and copy data
for i in df.index:
    for j in latlng.index:
        if df['PostalCode'][i]==latlng['Postal Code'][j]:
            df['Latitude'][i]=latlng['Latitude'][j]
            df['Longitude'][i]=latlng['Longitude'][j]

Let's create a map of Toronto:

In [5]:
# create map of New York using latitude and longitude values
latitude=43.6532
longitude=-79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, postalcode in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood'], df['PostalCode']):
    label = '{}, {}, {}'.format(postalcode, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [6]:
CLIENT_ID = 'IGUYUIX2FPIECYVBZZSGXDUIZV4NR0J5VH0YOYA40QTZNV5T' # your Foursquare ID
CLIENT_SECRET = '0YZU3TBNTTRQ0RULZRY4UQOZYSN0P1Y4CXFLRH22NGKHKAMT' # your Foursquare Secret
VERSION = '20181217'
LIMIT=30
# Foursquare API version

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## The clustering will be done according to the Borough so they could be seen easily in the map, but using the whole neighborhoods.

In [8]:
toronto_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [9]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

### Clustering  
5 clusters to separate different values so we could see which Boroughs are similar to each other, or if they are alone. 

In [10]:
dfborough=df.groupby(by='Borough').mean()
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
dfborough['Cluster Labels']=kmeans.labels_
dfborough.reset_index(inplace=True)
dfborough

,Borough,Latitude,Longitude,Cluster Labels
0,Central Toronto,43.701980,-79.398954,2
1,Downtown Toronto,43.654169,-79.383665,2
2,East Toronto,43.669436,-79.324654,2
3,East York,43.700303,-79.335851,2
4,Etobicoke,43.660043,-79.542074,0
5,Mississauga,43.636966,-79.615819,3
6,North York,43.750727,-79.429338,2
7,Queen's Park,43.662301,-79.389494,1
8,Scarborough,43.766229,-79.249085,2
9,West Toronto,43.652653,-79.449290,2


The map will be defined following the previous lab.

In [15]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9.5)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(dfborough['Latitude'], dfborough['Longitude'], dfborough['Borough'], dfborough['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

So it seems the closest boroughs to the center of Toronto are similar to each other than the ones closest to the suburbs. This analysis would be easily done by neighborhood by just clustering the neighborhoods instead of the boroughs.